In [1]:
import os
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import utils
import dataset
import keras.backend as K
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.layers import Input, Reshape
from keras.layers import Conv2D, MaxPooling2D, Activation , Flatten, Dense
from keras.layers import RNN, LSTM, GRU, ConvLSTM2D
from keras.layers import Deconv3D,UpSampling3D, Activation

Using TensorFlow backend.


In [2]:
# some magic commands
# %load_ext autoreload
%matplotlib inline

In [17]:
# load data
X=tf.convert_to_tensor(datasets.load_dataset("ShapeNet"),dtype=tf.float32)
Y=None;

[load_dataset] loading ShapeNet
[construct_paths] for dir ShapeNet
[fetch_renders_from_disk] in dir ./Renders
[fetch_renders_from_disk] returning 10 renders from ./Renders


In [18]:
# net parameters
conv_filter_count = [96, 128, 256, 256, 256, 256]
deconv_filter_count = [128, 128, 128, 64, 32, 2]

In [19]:
# encoder network
print(X.shape)
cur_tensor=X
k_s = [3,3]

for i in range(6): 
    ks=[7,7]if i is 0 else k_s  
    with K.name_scope("encoder_layer"):
        cur_tensor=Conv2D(filters=conv_filter_count[i],kernel_size= k_s)(cur_tensor)
        cur_tensor=MaxPooling2D()(cur_tensor)
        cur_tensor=Activation('relu')(cur_tensor)
    print(cur_tensor.shape)

(10, 512, 512, 3)
(10, 255, 255, 96)
(10, 126, 126, 128)
(10, 62, 62, 256)
(10, 30, 30, 256)
(10, 14, 14, 256)
(10, 6, 6, 256)


In [20]:
# construct input to 3D-LSTM cells
# cur_tensor=Flatten()(cur_tensor)
# cur_tensor
# print(cur_tensor.shape)
cur_tensor=Dense(1024)(cur_tensor)
print(cur_tensor.shape)
cur_tensor=Reshape([1,1024])(cur_tensor)
print(cur_tensor.shape)

(10, 6, 6, 1024)


ValueError: total size of new array must be unchanged

In [ ]:
# 3D-LSTM 
# construct 3D grid of cells
cell_grid= utils.grid3D(GRU(1024),N=3)
out=[];
with K.name_scope("3D-LSTM"):
    for cell in np.nditer(cell_grid,flags=['refs_ok']):
        out.append((cell.item())(cur_tensor));
        
out=np.array(out)

In [ ]:
cell_grid.shape

In [ ]:
print(type(out[0]))
print(out.shape)
out=out.reshape([3,3,3])
print(type(out[0,0,0]))
print(out.shape)

In [ ]:
# decoder network
for i in range(6):
    with K.name_scope("decoder_layer"):
        cur_tensor=Deconv3D(filters=deconv_filter_count[i],kernel_size=(3,3,3))(cur_tensor)
        cur_tensor=UpSampling3D()(cur_tensor)
        cur_tensor=Activation('relu')(cur_tensor)
        print(cur_tensor.shape)
    
print(cur_tensor.shape())

In [ ]:
net = Model(input=input_tensor,output=cur_tensor)
net.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
os.system("rm -rf ./logs/*")
tensor_board=TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
net.fit(X,np.random.rand(len(X),1,1024),epochs=1,callbacks=[tensor_board])